# DNN Model Notebook

```md
@authors: miguelrocha and Grupo 03
```

In [5]:
# Notebook Imports
import numpy as np

from models.dnn_model import NeuralNetwork, hyperparameter_optimization
from helpers.dataset import Dataset
from helpers.enums import ModelRunMode
from helpers.layers import DenseLayer
from helpers.activation import ReLUActivation, SigmoidActivation
from helpers.losses import BinaryCrossEntropy
from helpers.optimizer import Optimizer
from helpers.metrics import accuracy
from helpers.enums import ModelType
from helpers.model import save_dnn_model

In [6]:
# Model run mode
# Options: 
#   ModelRunMode.TRAIN.value            (Train the model)
#   ModelRunMode.CLASSIFY.value         (Classify data)
mode = ModelRunMode.TRAIN.value
# Prefix for saving the model files
model_prefix = "dnn_model_1"

In [7]:
# Parameters cell
if mode == ModelRunMode.TRAIN.value:
    # TRAIN mode: Set parameters for training
    input_csv = "../tarefa_1/clean_input_datasets/dataset1_enh_inputs.csv"              # CSV file with training inputs (ID, Text)
    output_csv = "../tarefa_1/clean_output_datasets/dataset1_enh_outputs.csv"           # CSV file with training outputs (ID, Label)
    test_size = 0.3                                                                     # Proportion of the dataset to use as test data
    validation_size = 0.2                                                               # Proportion of the dataset reserved as validation data (used during hyperparameter tuning)
    verbose = True                                                                      # Verbosity level enabler
    random_state=42                                                                     # Seed for reproducible dataset splitting
    max_vocab_size=None                                                                 # Maximum vocabulary size (None implies no limit)
    min_freq=1                                                                          # Minimum frequency required for a word to be included in the vocabulary
elif mode == ModelRunMode.CLASSIFY.value:
    # CLASSIFY mode: Set parameters for classification
    input_csv = "../tarefa_2/classify_input_datasets/dataset2_inputs.csv"               # CSV file with texts for prediction (ID, Text)
    output_csv = "../tarefa_2/classify_output_datasets/dataset2_outputs_dnn_model.csv"  # CSV file to store prediction result
else:
    print("The selected option is not valid. Options: \"train\" or \"classify\"!")
    SystemExit()

In [8]:
if mode == ModelRunMode.TRAIN.value:
    # Load datasets using TF-IDF vectorization
    X_train, y_train, X_test, y_test, vocab, idf = Dataset.prepare_train_test_tfidf(input_csv=input_csv, output_csv=output_csv, test_size=test_size, random_state=random_state, max_vocab_size=max_vocab_size, min_freq=min_freq)

    # Create Dataset objects for training and test data
    train_ds_full = Dataset(X=X_train, Y=y_train)
    test_ds = Dataset(X=X_test, Y=y_test)

    # Display dimensions of the training and test datasets
    print(f"Train set has {train_ds_full.nrows()} rows and {train_ds_full.ncols()} columns")
    print(f"Test set has {test_ds.nrows()} rows and {test_ds.ncols()} columns\n")

    # Split the full training dataset into training and validation subsets (80% training, 20% validation)
    n_train = train_ds_full.X.shape[0]
    indices = np.arange(n_train)
    np.random.shuffle(indices)
    split_idx = int((1 - validation_size) * n_train)
    train_idx = indices[:split_idx]
    val_idx = indices[split_idx:]
    train_ds = Dataset(X=train_ds_full.X[train_idx], Y=train_ds_full.Y[train_idx])
    val_ds = Dataset(X=train_ds_full.X[val_idx], Y=train_ds_full.Y[val_idx])

    # More varied epochs to handle both simpler and more complex tasks
    epochs_list = [40, 60, 80, 100, 120]

    # A wider range of batch sizes
    batch_size_list = [8, 16, 32]

    # Learning rate from very small to moderate
    learning_rate_list = [0.001, 0.01, 0.1]

    # Momentum slightly varied around typical defaults
    momentum_list = [0.85, 0.9, 0.95]

    # Different hidden layer topologies
    hidden_layers_list = [
        [32],
        [64],
        [64, 32],
        [128, 64]
    ]

    # Add mild to moderate dropout levels
    dropout_list = [0.0, 0.01, 0.1, 0.5]

    # Perform hyperparameter tuning using the training and validation sets
    print("Starting hyperparameter tuning...")
    best_params = hyperparameter_optimization(train_ds, val_ds, epochs_list, batch_size_list, learning_rate_list, momentum_list, hidden_layers_list, dropout_list, n_iter=15)
    print("\nBest hyperparameters:", best_params)

    # Retrain DNN model on the full training data using the best hyperparameters
    final_model = NeuralNetwork(epochs=best_params['epochs'], batch_size=best_params['batch_size'], optimizer=Optimizer(learning_rate=best_params['learning_rate'], momentum=best_params['momentum']), verbose=verbose, loss=BinaryCrossEntropy,  metric=accuracy)

    # Build model Layers
    n_features = train_ds.X.shape[1]
    for i, units in enumerate(best_params['n_hidden']):
        if i == 0:
            final_model.add(DenseLayer(units, (n_features,),dropout_rate=best_params['dropout_rate']))
        else:
            final_model.add(DenseLayer(units,dropout_rate=best_params['dropout_rate']))
        final_model.add(ReLUActivation())
        
    final_model.add(DenseLayer(1))
    final_model.add(SigmoidActivation())

    # Fit model
    final_model.fit(train_ds)
    out = final_model.predict(test_ds)

    # Evaluate the final model on the test set
    acc = final_model.score(test_ds, out)
    print("Test Accuracy:", acc)

    # Save the model, plus vocab & idf, so classification can replicate
    save_dnn_model(dnn=final_model, vocab=vocab, idf=idf, model_prefix=model_prefix)
    print(f"Model saved with prefix {model_prefix}")

if mode == ModelRunMode.CLASSIFY.value:
    # Classify new texts using the saved model
    Dataset.classify_texts(input_csv=input_csv, output_csv=output_csv, neural_net_class=NeuralNetwork, model_type=ModelType.DNN ,model_prefix=model_prefix)

Train set has 70 rows and 1928 columns
Test set has 30 rows and 1928 columns

Starting hyperparameter tuning...


Hyperparameter Search:   7%|▋         | 1/15 [00:00<00:11,  1.27it/s]

Parameters: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.9, 'n_hidden': [64, 32], 'dropout_rate': 0.1}  |  Accuracy: 0.5000
Parameters: {'epochs': 40, 'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.85, 'n_hidden': [32], 'dropout_rate': 0.01}  |  Accuracy: 0.3571


Hyperparameter Search:  20%|██        | 3/15 [00:01<00:05,  2.25it/s]

Parameters: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.001, 'momentum': 0.85, 'n_hidden': [64, 32], 'dropout_rate': 0.01}  |  Accuracy: 0.6429


...

Hyperparameter Search:  87%|████████▋ | 13/15 [00:07<00:00,  2.28it/s]

Parameters: {'epochs': 120, 'batch_size': 8, 'learning_rate': 0.01, 'momentum': 0.95, 'n_hidden': [32], 'dropout_rate': 0.01}  |  Accuracy: 0.6429


Hyperparameter Search: 100%|██████████| 15/15 [00:08<00:00,  1.87it/s]

Parameters: {'epochs': 80, 'batch_size': 8, 'learning_rate': 0.001, 'momentum': 0.9, 'n_hidden': [32], 'dropout_rate': 0.0}  |  Accuracy: 0.3571
Parameters: {'epochs': 120, 'batch_size': 32, 'learning_rate': 0.1, 'momentum': 0.85, 'n_hidden': [32], 'dropout_rate': 0.01}  |  Accuracy: 0.7143

Best Hyperparameters Found: {'epochs': 40, 'batch_size': 8, 'learning_rate': 0.1, 'momentum': 0.85, 'n_hidden': [128, 64], 'dropout_rate': 0.5}
Best Accuracy: 0.7857

Best hyperparameters: {'epochs': 40, 'batch_size': 8, 'learning_rate': 0.1, 'momentum': 0.85, 'n_hidden': [128, 64], 'dropout_rate': 0.5}
Epoch 1/40 - loss: 0.7043 - accuracy: 0.5536
Epoch 2/40 - loss: 0.7258 - accuracy: 0.4643
Epoch 3/40 - loss: 0.7212 - accuracy: 0.4464
Epoch 4/40 - loss: 0.7051 - accuracy: 0.5357


Epoch 5/40 - loss: 0.6548 - accuracy: 0.5714
Epoch 6/40 - loss: 0.6968 - accuracy: 0.5179
Epoch 7/40 - loss: 0.6961 - accuracy: 0.5179
Epoch 8/40 - loss: 0.6862 - accuracy: 0.5714
Epoch 9/40 - loss: 0.6721 - accuracy: 0.6071
Epoch 10/40 - loss: 0.6136 - accuracy: 0.6964
Epoch 11/40 - loss: 0.6353 - accuracy: 0.5893
Epoch 12/40 - loss: 0.5686 - accuracy: 0.7321
Epoch 13/40 - loss: 0.6062 - accuracy: 0.7500
Epoch 14/40 - loss: 0.5910 - accuracy: 0.6786
Epoch 15/40 - loss: 0.5809 - accuracy: 0.6786
Epoch 16/40 - loss: 0.5920 - accuracy: 0.7500
Epoch 17/40 - loss: 0.5283 - accuracy: 0.7143
Epoch 18/40 - loss: 0.5272 - accuracy: 0.7321
Epoch 19/40 - loss: 0.4624 - accuracy: 0.8036
Epoch 20/40 - loss: 0.4511 - accuracy: 0.8036
Epoch 21/40 - loss: 0.4551 - accuracy: 0.8036
Epoch 22/40 - loss: 0.4094 - accuracy: 0.8393
Epoch 23/40 - loss: 0.4399 - accuracy: 0.7679
Epoch 24/40 - loss: 0.4718 - accuracy: 0.7321
Epoch 25/40 - loss: 0.3974 - accuracy: 0.8393
Epoch 26/40 - loss: 0.3125 - accuracy: 